In [88]:
! pip install ta

Error processing line 1 of C:\Users\alver\anaconda3\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\alver\anaconda3\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 562, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


In [89]:
#Technical Indicators
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from ta.volatility import AverageTrueRange
from ta.trend import SMAIndicator
from ta.trend import EMAIndicator
from ta.momentum import RSIIndicator
from ta.volume import VolumeWeightedAveragePrice

# Plotting
import matplotlib.pyplot as plt

#Getting df
import yfinance as yf
import math
import pandas as pd
import numpy as np

In [90]:
# MAANG stocks list
stocks = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL']
start_date = '2015-01-01'
end_date = '2019-12-31'

Signal and correlation test

In [ ]:
def MA200_Price_indicator(stock, start, end):
    df = yf.Ticker(stock).history(start = start, end = end)

    sma_200 = SMAIndicator(close = df['Close'], window = 200, fillna = False)
    df['sma_200'] = sma_200.sma_indicator()
    
    df["Position"] = np.where(df['sma_200'] < df['Close'], 1, -1)
    df["Position"] = np.where(pd.isna(df['sma_200']), 0, df['Position'])
    df['Position'] = df['Position'].fillna(0)

    return df


In [91]:
def EMA200_Price_indicator(stock, start, end):
    df = yf.Ticker(stock).history(start=start, end=end)

    df['ema_200'] = df['Close'].ewm(span=200, adjust=False).mean()
    
    df["Position"] = np.where(df['ema_200'] < df['Close'], 1, -1)
    
    df["Position"] = np.where(pd.isna(df['ema_200']), 0, df['Position'])
    
    df["Position"] = np.where(pd.isna(df['ema_200']), 0, df["Position"])

    return df

In [93]:
def RSI_indicator(stock, start, end, rsi_window, high_band, low_band):
    df = yf.Ticker(stock).history(start = start, end = end)

    rsi = RSIIndicator(close = df['Close'], window = rsi_window, fillna = False)
    df['RSI'] = rsi.rsi()

    df['Position'] = np.where(df['RSI'] > high_band, -1, np.where(df['RSI'] < low_band, 1, 0))

    return df

In [95]:
def BB_indicator(stock, start, end, bb_window):
    df = yf.Ticker(stock).history(start = start, end = end)

    bb = BollingerBands(close = df['Close'], window = bb_window, fillna = False)
    df['bb_high'] = bb.bollinger_hband()
    df['bb_mid'] = bb.bollinger_mavg()
    df['bb_low'] = bb.bollinger_lband()

    df['Position'] = np.where((df['bb_high'] > df['Close']) & (df['bb_low'] < df['Close']), 
                          0, 
                          np.where(df['bb_low'] > df['Close'], 1, -1))
    
    df.loc[df['bb_high'].isna() | df['bb_low'].isna(), 'Position'] = 0

    return df
    

In [97]:
def MACD_indicator(stock, start, end):
    df = yf.Ticker(stock).history(start = start, end = end)
    
    # Calculate the 200-day EMA
    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()
    
    # Calculate the MACD (12, 26, 9)
    exp12 = df['Close'].ewm(span=12, adjust=False).mean()
    exp26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp12 - exp26
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

    df['Position'] = np.where(df['Signal_Line'] < df['MACD'], 1, -1)

    return df

Correlation for signal for stocks

In [113]:
total_corr = pd.DataFrame()
for stock in stocks:
    sma_200_indicator = MA200_Price_indicator(stock, start_date, end_date)
    ema_200_indicator = EMA200_Price_indicator(stock, start_date, end_date)
    rsi_indicator = RSI_indicator(stock, start_date, end_date, 14, 70, 30)
    bb_indicator = BB_indicator(stock, start_date, end_date, 20)
    macd_indicator = MACD_indicator(stock, start_date, end_date)
    
    all_indicators = pd.DataFrame({
        'sma_signal' : sma_200_indicator['Position'],
        'ema_signal' : ema_200_indicator['Position'],
        'rsi_signal' : rsi_indicator['Position'],
        'bb_signal' : bb_indicator['Position'],
        'macd_signal' : macd_indicator['Position']
    })

    corr_matrix = all_indicators.corr()
    if total_corr.empty:
        total_corr = corr_matrix
    else:
        total_corr = total_corr.add(corr_matrix)

total_corr /= len(stocks)
    
print(f'{total_corr} \n')

             sma_signal  ema_signal  rsi_signal  bb_signal  macd_signal
sma_signal     1.000000    0.741284   -0.184050  -0.110197    -0.004338
ema_signal     0.741284    1.000000   -0.225322  -0.161457     0.052739
rsi_signal    -0.184050   -0.225322    1.000000   0.367405    -0.286779
bb_signal     -0.110197   -0.161457    0.367405   1.000000    -0.317001
macd_signal   -0.004338    0.052739   -0.286779  -0.317001     1.000000 

